이 튜토리얼에서는 NVIDIA NIM, LangChain, 그리고 Google Search API를 사용하여 RAG(Retrieval-Augmented Generation) 애플리케이션을 만드는 방법을 알아보겠습니다. 이 애플리케이션은 사용자의 질문에 대해 인터넷에서 관련 정보를 검색하고, 이를 바탕으로 정확하고 최신의 답변을 생성합니다.

#### 환경설치

In [ ]:
!pip install html2text

In [ ]:
!pip install chromadb

In [ ]:
!pip install langchain-nvidia-ai-endpoints langchain-community langchain-elasticsearch

In [ ]:
!pip install sentence-transformers

#### NIM 구축 및 배포
00_NIM.ipynb 참고

#### Google Custom Search Engine ID 생성
1. https://programmablesearchengine.google.com/controlpanel/create 에 접속뒤에 새 검색엔진 만들기 페이지에서 필요한 정보를 입력
2.만들기 클릭하여 검색엔진 생성
3. 검색 엔진 ID 확인 이 아이디를 Custom Search Engine ID로 사용

#### Google API key 생성

1. https://console.cloud.google.com/apis/credentials?pli=1 에 접속 뒤에 왼쪽 메뉴에서 "API 및 서비스" 버튼을 클릭하고 "API key" 를 선택
2. API 키가 생성되면 팝업창에 표시, 복사하여 보관 하거나 사용



#### Google Custome Search Engine ID 및 Google API Key 설정

In [ ]:
google_cse_id = ""

In [ ]:
google_api_key = ""

In [ ]:
import os
os.environ["GOOGLE_CSE_ID"] = google_cse_id
os.environ["GOOGLE_API_KEY"] = google_api_key

#### Goolge Serach engine 테스트

In [ ]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

In [ ]:
tool.run("역대 대한민국 대통령 이름은?")

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.vectorstores import Chroma
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.chains import RetrievalQAWithSourcesChain

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
import os
base_url = input("NIM URL을 입력하세요")

os.environ["NIM_URL"] = base_url

#### Vector DB 설정
- 한국어 임베딩 모델 사용 (https://huggingface.co/jhgan/ko-sroberta-multitask)

In [ ]:
vectorstore = Chroma(embedding_function=HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask'),
                     persist_directory="./chroma_db_oai")

#### Search Query를 위한 NIM

In [ ]:
search_llm = ChatNVIDIA(base_url="{0}/v1".format(os.environ["NIM_URL"]), api_key="not-used",temperature= 0,top_p= 1,max_token=1024)

#### Search API wrapper 생성

In [ ]:
search = GoogleSearchAPIWrapper()

#### Web Research Retriever 생성

In [ ]:
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=search_llm,
    search=search
)

In [ ]:
response_llm  = ChatNVIDIA(base_url="{0}/v1".format(os.environ["NIM_URL"]), api_key="not-used",temperature= 0,top_p= 1,max_token=1024)

In [ ]:
template = """다음은 사용자의 질문에 대한 Google 검색 결과입니다. 이 정보를 바탕으로 질문에 답변해 주세요.

질문: {question}

검색 결과:
{context}

위의 정보를 바탕으로 사용자의 질문에 대해 명확하고 정확하게 답변해 주세요. 검색 결과에 없는 정보는 추측하지 말고, 모르는 경우 솔직히 모른다고 답변하세요.
Answer should be Korean
답변:
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [ ]:
chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "context"
}

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(response_llm,
                                                       retriever=web_research_retriever,
                                                       chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input = input("사용자: ")

    if user_input.lower() == 'quit':
        print("종료합니다. 감사합니다!")
        break

    result = qa_chain({"question": user_input})
    print("답변:", result["answer"])